# A Tri-Stage Wrapper-Filter Feature Selection Framework for Disease Classification [Implmentation]

In [1]:
import numpy as np
import itertools
import pandas as pd
from functools import cmp_to_key
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.naive_bayes import CategoricalNB
from statistics import mean
from sklearn.ensemble import GradientBoostingClassifier
import random
import math # cos() for Rastrigin
import copy # array-copying convenience
import sys # max float
import numpy as np

In [2]:
NUM_M = 50 
NUM_K = 60
NUM_J = 23
MAX_R = 0.7
POPULATION_NUMBER = 70
ALPAH = 0.9
BETA = 1 - ALPAH

In [3]:
!pip install ReliefF

You should consider upgrading via the '/Users/guyarieli/opt/anaconda3/bin/python -m pip install --upgrade pip' command.


### Load Data

In [4]:
inner_func = [ (f'F{idx+1}R',f'F{idx+1}S') for idx in range(22)]
columns = ['OVERALL_DIAGNOSIS'] + list(itertools.chain(*inner_func))

In [5]:
with open('./Data/SPECTF.train') as f:
    
    lines = f.readlines()
    data_array = np.zeros((len(lines),45))
    for row_idx,line in enumerate(lines):
        data_array[row_idx] = list(map(lambda x: int(x),line.replace('\n','').split(',')))

In [6]:
data_dict = {col_name: data for col_name, data in zip(columns,data_array)}
data = pd.DataFrame.from_dict(data_dict)
data.head()

OVERALL_DIAGNOSIS   F1R   F1S   F2R   F2S   F3R   F3S   F4R   F4S   F5R  \
0                1.0   1.0   1.0   1.0   1.0   1.0   1.0   1.0   1.0   1.0   
1               59.0  72.0  71.0  69.0  70.0  57.0  69.0  61.0  65.0  74.0   
2               52.0  62.0  62.0  71.0  66.0  69.0  66.0  60.0  62.0  73.0   
3               70.0  69.0  70.0  70.0  61.0  68.0  62.0  60.0  67.0  72.0   
4               67.0  67.0  64.0  78.0  66.0  75.0  75.0  62.0  68.0  79.0   

   ...  F18R  F18S  F19R  F19S  F20R  F20S  F21R  F21S  F22R  F22S  
0  ...   1.0   1.0   1.0   1.0   1.0   0.0   0.0   0.0   0.0   0.0  
1  ...  71.0  70.0  73.0  68.0  68.0  62.0  62.0  59.0  75.0  77.0  
2  ...  75.0  66.0  76.0  76.0  64.0  67.0  67.0  68.0  75.0  79.0  
3  ...  76.0  66.0  68.0  79.0  65.0  64.0  68.0  69.0  70.0  79.0  
4  ...  74.0  68.0  74.0  78.0  68.0  70.0  70.0  67.0  77.0  77.0  

[5 rows x 45 columns]

In [7]:
X,y = data.iloc[:,1:],data.iloc[:,0]

### Ranked Algorithm

In [8]:
# Mutual Infomation
from sklearn.feature_selection import mutual_info_classif as MI
from sklearn.feature_selection import chi2 as CS
from ReliefF import ReliefF

In [9]:
def XVariance(X,y,top_m):
    X_=X[:].apply(pd.to_numeric, errors='coerce')
    #X_var.dtypes
    X_var = (X_).var()
    #print (X_var)
    Y_ = y[:].apply(pd.to_numeric, errors='coerce')
    Y_var=Y_.var()
    D = X_var + Y_var
    DL=D.nlargest(n=top_m)
    return np.array(DL.index)

### Articture

<img src="Assets/articture.png" alt="drawing" style="width:500px;"/>


 We'll start by buolding diffrent phases sequentially

### PHASE 1

In [10]:
from sklearn.feature_selection import mutual_info_classif as MI
from sklearn.feature_selection import chi2 as CS
from ReliefF import ReliefF

In [11]:
def score_to_feature_name(columns_name,score):
    score = list(zip(columns_name,score))
    score = sorted(score, key=cmp_to_key(lambda item1,item2: item2[1]-item1[1]))
    return score

In [12]:
# Get mutual information
mi_top_m = [feature for feature, score in score_to_feature_name(X.columns,MI(X,y))[-NUM_M:]]
# Get Chi Square 
cs_top_m = [feature for feature, score in score_to_feature_name(X.columns,CS(X,y)[1])[-NUM_M:]]
# Get Xvariance 
cs_top_m = XVariance(X,y,NUM_M)
# Get RFF 
relief_data = ReliefF(n_neighbors=20, n_features_to_keep=NUM_M).fit_transform(X.values,y.values)
rff_top_m = []
for col_name in X.columns:
    col_vector = X[col_name]
    for rff_vector in relief_data.T:
        if np.all(col_vector==rff_vector):
            rff_top_m.append(col_name)
            break;
# Union-set 
features_selected = set([*mi_top_m,*cs_top_m,*rff_top_m,*cs_top_m])

In [13]:
def activate_model_on_features(features,X,y,model_class,model_kwargs):
    score_vector = np.zeros(X.shape[0])
    for idx, feature in enumerate(features):
        train = X[feature].to_numpy().reshape(1,-1).T
        test = y
        model = model_class(**model_kwargs).fit(train, test)
        score_vector[idx] = model.score(train,test)
    return score_vector

In [14]:
def score_knn_nb_svm(features_set,X,y):
    score = activate_model_on_features(features_set,X,y,KNeighborsClassifier,{'n_neighbors':4})
    score += activate_model_on_features(features_set,X,y,SVC,{'gamma':'auto'})
    score += activate_model_on_features(features_set,X,y,CategoricalNB,{})
    return score

In [34]:
def get_xgb_top_k(top_k_features, X, y):
    xgboost = GradientBoostingClassifier(n_estimators=100, learning_rate=1.0,max_depth=1, random_state=0).fit(X, y)
    prev_accrucy = 0.
    for idx in range(len(top_k_features)):
        k_selected_feature_names = top_k_features[:idx+1]
        data = X.loc[:,k_selected_feature_names]
        current_accrucy = xgboost.fit(data,y).score(data,y)
        if len(k_selected_feature_names) >= NUM_J:
            break
    return top_k_features[:idx]

In [36]:
score = score_knn_nb_svm(features_selected,X,y)
top_k_features = [feature for feature, score in set(score_to_feature_name(X.columns,score))]
top_k_features = get_xgb_top_k(top_k_features,X,y)


### PHASE 2

In [37]:
from sklearn.feature_selection import r_regression as PCC

In [38]:
rmv_set = set()
seen_set = set()
class_val = y.to_numpy().reshape(1,-1).ravel()
for  _to, val1 in enumerate(top_k_features):
    if val1 in seen_set:
        continue
    for val2 in top_k_features[:_to+1]:
        if val2 in seen_set or val2 in rmv_set:
            continue
        val1_data = X[val1].to_numpy().reshape(1,-1).T
        val2_data = X[val2].to_numpy()
        if abs(int(PCC(val1_data,val2_data))) >= MAX_R:
            val2_data = val2_data.reshape(1,-1).T
            rmv_val = val1 if abs(int(PCC(val1_data,class_val))) > abs(int(PCC(val2_data,class_val))) else val2
            rmv_set.add(rmv_val)
        seen_set.add(val2)
    seen_set.add(val1)
print(top_k_features)
uncorrelated_k = set(top_k_features)-rmv_set

['F17R', 'F1S', 'F9R', 'F11R', 'F8S', 'F22R', 'F5R', 'F6R', 'F19R', 'F7R', 'F13S', 'F4R', 'F16R', 'F12R', 'F7S', 'F19S', 'F21R', 'F21S', 'F8R', 'F3S', 'F5S', 'F1R']


In [39]:
top_j_features = get_xgb_top_k(list(uncorrelated_k),X,y)

### PHASE 3

<img src="Assets/WOA.png" alt="drawing" style="width:500px;"/>

In [40]:
class WOA:

    def __init__(self, fitness_func, limit, population_n,sol_dim, seed,max_type=True):
        self._fitness_func = fitness_func
        self._limit = limit
        self._population_n = population_n
        self._sol_dim = sol_dim
        self._max_type = max_type
        np.random.seed(seed)
        self.rnd = random.Random(seed)
    
    def _init_population(self):
        self._population = np.random.uniform(self._limit[0],self._limit[1],size=(self._population_n,self._sol_dim))

    def _get_best_sol_index(self):
        after_fitness = np.array(list(map(self._fitness_func,self._population)))
        return np.argmax(after_fitness) if self._max_type else np.argmin(after_fitness)

    def run(self,max_iter=20):
        self._init_population()
        min_bound = np.array([self._limit[0]]*self._sol_dim,dtype=np.float32)
        max_bound = np.array([self._limit[1]]*self._sol_dim,dtype=np.float32)
        for p in range(max_iter):
            x_star_index = self._get_best_sol_index()
            x_star = np.copy(self._population[x_star_index])
            s = 2 * (1 - p / max_iter)
            s2 = -1 + p *(-1/max_iter)
            b = 1
            l = (s2-1)*self.rnd.random()+1;
            for idx,X in enumerate(self._population):
                V = self.rnd.random()
                K = 2 * s * V - s
                J = 2 * V
                t = self.rnd.random()
                B = abs(J*x_star-X)
                if t < .5:
                    if abs(K) < 1:
                        self._population[idx] = x_star - K*B
                    else:
                        option_lst = set([idx for idx in range(self._population_n)]) - {p}
                        xr = random.choice(list(option_lst))
                        self._population[idx] = xr-K*B
                else:
                    self._population[idx] = x_star + B * math.exp(b * l) * math.cos(2 * math.pi * l)
                # Make sure all value in bound range
                self._population[idx] = np.maximum(min_bound,self._population[idx])
                self._population[idx] = np.minimum(max_bound,self._population[idx])
        return np.rint(x_star).astype(int)

In [41]:
feature_mapper = {idx:name for idx,name in enumerate(X.columns)}

In [42]:
def get_features_by_bit_vector(vector,feature_mapper):
    feature_vec = []
    for idx,val in enumerate(vector):
        if val >= 0.5:
            feature_vec.append(feature_mapper[idx])
    return np.array(feature_vec)

In [43]:
def paper_fitness(sol):
    C = len(sol)
    acc_rate = 0
    R = len(sol[sol >= 0.5])
    if R > 0 : 
        sol_features = get_features_by_bit_vector(sol,feature_mapper)
        train = X.loc[:,sol_features].to_numpy()
        acc_rate += KNeighborsClassifier(n_neighbors=3).fit(train,y).score(train,y)
    feature_part = BETA * (C-R / C)
    score_gamma = acc_rate * ALPAH
    return score_gamma + feature_part
    

In [44]:
sol_len = len(top_j_features)
print(f" Whale size: {sol_len}")
woa = WOA(paper_fitness, [0,1], POPULATION_NUMBER, sol_len, 123,max_type=True)

 Whale size: 4


In [45]:
solution = woa.run()

In [46]:
best_features = get_features_by_bit_vector(solution,feature_mapper)
best_features

array(['F2R', 'F2S'], dtype='<U3')